In [4]:
#!pip install surprise

In [3]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from collections import defaultdict
from surprise.model_selection import train_test_split
import io 

from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [5]:
path = 'clustered_clients.csv'
df = pd.read_csv(path)

In [6]:
#add id for each client
path_id = 'vitrina_clients_vkr.xlsx'
with open(path_id, 'rb') as f:
    df_initial = pd.read_excel(io.BytesIO(f.read()), sheet_name='Sheet1', nrows=200,engine='openpyxl')


# df_initial = pd.read_excel(path, sheet_name='Sheet1', nrows=200,engine='openpyxl')
id_col = df_initial['client_id']


In [7]:
df['client_id'] = id_col

In [8]:
df = df[:10000]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DistinctCountOrders        10000 non-null  int64  
 1   AverageOrderSum            10000 non-null  float64
 2   AvgBill                    10000 non-null  float64
 3   AverageNumItemsInOrder     10000 non-null  float64
 4   RevenueClient              10000 non-null  float64
 5   TotalNumberItemsOrdered    10000 non-null  int64  
 6   AverageMarginOrder         10000 non-null  float64
 7   cnt_purchased_orders       10000 non-null  int64  
 8   AvgOrderSumPurchased       10000 non-null  float64
 9   AvgNumItemsOrderPurchased  10000 non-null  float64
 10  RevenueClientPurchased     10000 non-null  float64
 11  PurchasedItems             10000 non-null  float64
 12  AvgMarginPurchased         10000 non-null  float64
 13  SharePurchasedOrders       10000 non-null  floa

In [ ]:
# df.to_csv('hybrid_rc/data/processed/clustered_clients200.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DistinctCountOrders        10000 non-null  int64  
 1   AverageOrderSum            10000 non-null  float64
 2   AvgBill                    10000 non-null  float64
 3   AverageNumItemsInOrder     10000 non-null  float64
 4   RevenueClient              10000 non-null  float64
 5   TotalNumberItemsOrdered    10000 non-null  int64  
 6   AverageMarginOrder         10000 non-null  float64
 7   cnt_purchased_orders       10000 non-null  int64  
 8   AvgOrderSumPurchased       10000 non-null  float64
 9   AvgNumItemsOrderPurchased  10000 non-null  float64
 10  RevenueClientPurchased     10000 non-null  float64
 11  PurchasedItems             10000 non-null  float64
 12  AvgMarginPurchased         10000 non-null  float64
 13  SharePurchasedOrders       10000 non-null  floa

In [11]:
for col in df.columns:
    if type(df[col][0]) == np.float64:
        df[col] = df[col].astype('float16')
    elif type(df[col][0]) == np.int64:
        df[col] = df[col].astype('int16')
    elif type(df[col][0]) == np.object:
        df[col] = df[col].astype('category')
    else:
        pass

<ipython-input-11-0d2de1d4dbec>:6: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif type(df[col][0]) == np.object:


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DistinctCountOrders        10000 non-null  int16  
 1   AverageOrderSum            10000 non-null  float16
 2   AvgBill                    10000 non-null  float16
 3   AverageNumItemsInOrder     10000 non-null  float16
 4   RevenueClient              10000 non-null  float16
 5   TotalNumberItemsOrdered    10000 non-null  int16  
 6   AverageMarginOrder         10000 non-null  float16
 7   cnt_purchased_orders       10000 non-null  int16  
 8   AvgOrderSumPurchased       10000 non-null  float16
 9   AvgNumItemsOrderPurchased  10000 non-null  float16
 10  RevenueClientPurchased     10000 non-null  float16
 11  PurchasedItems             10000 non-null  float16
 12  AvgMarginPurchased         10000 non-null  float16
 13  SharePurchasedOrders       10000 non-null  floa

In [13]:
df.head()

,DistinctCountOrders,AverageOrderSum,AvgBill,AverageNumItemsInOrder,RevenueClient,TotalNumberItemsOrdered,AverageMarginOrder,cnt_purchased_orders,AvgOrderSumPurchased,AvgNumItemsOrderPurchased,...,ППКП,CENTRAL,FAR EAST,NORTH,PRIVOLZIE,SIBERIA,SOUTHERN,URAL,Cluster,client_id
0,100,1759.0,1723.0,2.439453,inf,244,219.0000,80,1575.0,2.275391,...,0.024597,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Cluster 3,55575053-54505550565678
1,66,580.5,524.0,3.787109,34592.0,250,144.1250,60,542.0,2.867188,...,0.459961,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Cluster 1,55575048-48524956565375
2,54,823.0,807.0,2.703125,43584.0,146,95.8125,53,816.0,2.679688,...,0.006851,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Cluster 1,55575555-57524953505673
3,59,3608.0,3482.0,4.167969,inf,246,429.0000,45,3218.0,2.222656,...,0.105713,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Cluster 3,55574953-50505150484978
4,46,2670.0,2632.0,1.717773,inf,79,310.7500,40,2690.0,1.799805,...,0.088623,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Cluster 3,55575054-51515151484875


In [14]:
print(df.head())

   DistinctCountOrders  AverageOrderSum  AvgBill  AverageNumItemsInOrder  \
0                  100           1759.0   1723.0                2.439453   
1                   66            580.5    524.0                3.787109   
2                   54            823.0    807.0                2.703125   
3                   59           3608.0   3482.0                4.167969   
4                   46           2670.0   2632.0                1.717773   

   RevenueClient  TotalNumberItemsOrdered  AverageMarginOrder  \
0            inf                      244            219.0000   
1        34592.0                      250            144.1250   
2        43584.0                      146             95.8125   
3            inf                      246            429.0000   
4            inf                       79            310.7500   

   cnt_purchased_orders  AvgOrderSumPurchased  AvgNumItemsOrderPurchased  ...  \
0                    80                1575.0                   2.27539

In [15]:
print(df.dtypes)

DistinctCountOrders            int16
AverageOrderSum              float16
AvgBill                      float16
AverageNumItemsInOrder       float16
RevenueClient                float16
TotalNumberItemsOrdered        int16
AverageMarginOrder           float16
cnt_purchased_orders           int16
AvgOrderSumPurchased         float16
AvgNumItemsOrderPurchased    float16
RevenueClientPurchased       float16
PurchasedItems               float16
AvgMarginPurchased           float16
SharePurchasedOrders         float16
ДЕТСКОЕ ПИТАНИЕ              float16
ЖЕНСКИЕ ШТУЧКИ               float16
ИГРУШКИ                      float16
КАНЦТОВАРЫ, КНИГИ, ДИСКИ     float16
КОСМЕТИКА/ГИГИЕНА            float16
КРУПНОГАБАРИТНЫЙ ТОВАР       float16
ОБУВЬ                        float16
ПОДГУЗНИКИ                   float16
СОПУТСТВУЮЩИЕ ТОВАРЫ         float16
ТЕКСТИЛЬ, ТРИКОТАЖ           float16
ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА    float16
ТОВАРЫ ДЛЯ ЖИВОТНЫХ          float16
ТОВАРЫ ДЛЯ КОРМЛЕНИЯ         float16
И

In [16]:
choose_column = ['AverageOrderSum']
selected_columns = ['client_id', 'Cluster'] + choose_column

In [17]:
selected_columns

['client_id', 'Cluster', 'AverageOrderSum']

In [18]:
maxi = df[choose_column].max().max()
mini = df[choose_column].min().min()

reader = Reader(rating_scale=(mini, maxi))

In [ ]:
# df[selected_columns] = df[selected_columns][:10]

In [19]:
df[selected_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   client_id        200 non-null    object 
 1   Cluster          10000 non-null  object 
 2   AverageOrderSum  10000 non-null  float16
dtypes: float16(1), object(2)
memory usage: 175.9+ KB


In [20]:
data = Dataset.load_from_df(df[selected_columns], reader)

In [21]:
data

In [22]:
trainset, testset = train_test_split(data, test_size=.25)
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)

In [23]:
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [24]:
predictions = algo.test(testset)

In [25]:
#predictions to dataframe
predictions_df = pd.DataFrame(predictions)

In [26]:
predictions_df

,uid,iid,r_ui,est,details
0,NaN,Cluster 0,2416.0,1689.39375,"{'actual_k': 40, 'was_impossible': False}"
1,NaN,Cluster 2,1877.0,3272.10000,"{'actual_k': 40, 'was_impossible': False}"
2,NaN,Cluster 1,5040.0,7256.65000,"{'actual_k': 40, 'was_impossible': False}"
3,NaN,Cluster 0,1012.5,1689.39375,"{'actual_k': 40, 'was_impossible': False}"
4,NaN,Cluster 0,1796.0,1689.39375,"{'actual_k': 40, 'was_impossible': False}"
...,...,...,...,...,...
2495,NaN,Cluster 2,4360.0,3272.10000,"{'actual_k': 40, 'was_impossible': False}"
2496,NaN,Cluster 2,3330.0,3272.10000,"{'actual_k': 40, 'was_impossible': False}"
2497,NaN,Cluster 0,1532.0,1689.39375,"{'actual_k': 40, 'was_impossible': False}"
2498,NaN,Cluster 1,6280.0,7256.65000,"{'actual_k': 40, 'was_impossible': False}"


In [27]:
#show similar users
algo.get_neighbors(1, k=10)

[3, 2, 0]

In [ ]:
#show unique users
print(df['client_id'].unique())

['55575053-54505550565678' '55575048-48524956565375'
 '55575555-57524953505673' '55574953-50505150484978'
 '55575054-51515151484875' '55575450-57505256495470'
 '55575054-50505252535072' '55574857-57555250515173'
 '55575448-56525355485670' '55575653-54505352545577'
 '55574951-50524849525479' '55574854-55565252495471'
 '55574954-56525352555270' '55574851-52514853535175'
 '55574850-53545452504975' '55575054-55485350545475'
 '55574851-49545557505179' '55575456-56525152515570'
 '55575050-57534855495577' '55574856-51575348534875'
 '55575653-51534851495771' '55575055-48485751575573'
 '55575054-57494849545177' '55575054-49525757494873'
 '55574857-48534848515278' '55575555-51575255495676'
 '55575155-49565654515772' '55574954-56485655505473'
 '55574948-48575750575679' '55574854-55554856495371'
 '55575451-54485257525274' '55575349-54575655495273'
 '55574954-57484849515770' '55575057-54555151565576'
 '55575555-51505357535179' '55575451-54575349555476'
 '55575351-53535250495170' '55575148-554948564

In [28]:
for i in range(10):
    try:
        client = df['client_id'].unique()[i]
        print("client: ", client, "cluster: ", df[df['client_id'] == client]['Cluster'].values[0], "prediction: ",algo.get_neighbors(i, k=10))
    except Exception as e:
        print(e)
        pass


client:  55575053-54505550565678 cluster:  Cluster 3 prediction:  [2, 1, 3]
client:  55575048-48524956565375 cluster:  Cluster 1 prediction:  [3, 2, 0]
client:  55575555-57524953505673 cluster:  Cluster 1 prediction:  [0, 1, 3]
client:  55574953-50505150484978 cluster:  Cluster 3 prediction:  [1, 2, 0]
index 4 is out of bounds for axis 0 with size 4
index 5 is out of bounds for axis 0 with size 4
index 6 is out of bounds for axis 0 with size 4
index 7 is out of bounds for axis 0 with size 4
index 8 is out of bounds for axis 0 with size 4
index 9 is out of bounds for axis 0 with size 4


In [29]:
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

In [30]:
benchmark = []
# Iterate over all algorithms

algorithms = [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]

print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    # results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=3, verbose=False)

    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')


Attempting:  [<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7f4081cacf10>, <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7f4081cacbe0>, <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x7f4081cac9a0>, <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7f4081cacb20>, <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7f4081cac9d0>, <surprise.prediction_algorithms.knns.KNNBaseline object at 0x7f4081cacb80>, <surprise.prediction_algorithms.knns.KNNBasic object at 0x7f4081cacc70>, <surprise.prediction_algorithms.knns.KNNWithMeans object at 0x7f4081caca00>, <surprise.prediction_algorithms.knns.KNNWithZScore object at 0x7f4081cac5e0>, <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x7f4081cac340>, <surprise.prediction_algorithms.co_clustering.CoClustering object at 0x7f4081cac550>] 



Starting:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7f4081c

In [31]:
print(pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse'))

                    test_rmse    fit_time  test_time
Algorithm                                           
SlopeOne          1656.440620    1.508909  11.426521
KNNWithZScore     1663.338897    0.270297   4.518291
KNNBasic          1666.018831    0.245512   4.396290
CoClustering      1675.692480    0.059379   0.023667
KNNWithMeans      1687.625716    0.231792   4.673701
KNNBaseline       1714.155064    0.266488   4.523497
BaselineOnly      1737.442700    0.003916   0.009624
NormalPredictor   3101.842893    0.005664   0.027784
NMF               3548.368961    0.065093   0.018592
SVDpp            43334.581284  113.350895  19.051319
SVD              43334.590330    0.164478   0.036649


In [32]:
#recommendation for client
client = df['client_id'].unique()[0]
print("client: ", client, "cluster: ", df[df['client_id'] == client]['Cluster'].values[0], "prediction: ",algo.get_neighbors(0, k=10))


client:  55575053-54505550565678 cluster:  Cluster 3 prediction:  [2, 1, 3]


In [33]:
#recommendation for client
client = df['client_id'].unique()[1]
print("client: ", client, "cluster: ", df[df['client_id'] == client]['Cluster'].values[0], "prediction: ",algo.get_neighbors(1, k=10))


client:  55575048-48524956565375 cluster:  Cluster 1 prediction:  [3, 2, 0]
